#### 1. Import library

In [2]:
import os
import pandas as pd 
import numpy as np 
from typing import Dict, List, Tuple 
import sqlite3
import json
import glob

#### 2. Load the data onto DataFrame

In [6]:
df_data = pd.DataFrame()
file_list = ['../data/' + i for i in os.listdir('../data/')]
dfs = [] # an empty list to store the data frames
file_list;

In [7]:
# load the json file of one month onto dataframe 
# df_data = pd.read_json('../data/airlines-1558527599826.json', lines=True)
for file in file_list:
    try:
        data = pd.read_json(file, lines=True) # read data frame from json file 
        print(file)
        dfs.append(data) # append the data frame to the list
    except:
        pass
df_data = pd.concat(dfs, ignore_index=True) # concatenate all the data frames in the list.

../data/airlines-1558527599826.json
../data/airlines-1558546003827.json
../data/airlines-1558678330070.json
../data/airlines-1558697205154.json
../data/airlines-1558744391657.json
../data/airlines-1558779687636.json
../data/airlines-1558863520888.json
../data/airlines-1558888297881.json
../data/airlines-1558974571041.json
../data/airlines-1558998029487.json
../data/airlines-1559069822287.json
../data/airlines-1559097885748.json
../data/airlines-1559142170524.json
../data/airlines-1559156713030.json
../data/airlines-1559231904332.json
../data/airlines-1559256655329.json
../data/airlines-1559323725358.json
../data/airlines-1559352498975.json


In [75]:
df_datas = df_data.dropna(subset = ["user"]).reset_index()

In [79]:
#Add the user_id attribute
df_user = [df_datas.user.iloc[i].get('id') for i in range(len(df_datas))]
df_datas['user_id'] = pd.Series(df_user)

In [80]:
df_datas.to_csv('../database/CSV file/general_tweets.csv')

#### 3. Store the data on .db file 

In [9]:
# create connection and cursor 
con = sqlite3.connect('../database/sqlite-tools-win32-x86-3380300/database.db')
cur = con.cursor()

#### 4. KLM table 

In [10]:
#Extract only tweets made by KLM (including posting +replying to other):
test = df_datas.copy()

ind = []
for index, row in df_datas.iterrows():
    if row['user']['id_str'] == '56377143':
        ind.append(index)
klm_tweet = df_datas.loc[ind]

In [11]:
#Extract only tweets mention KLM (@KLM etc...):
test2 = df_datas.copy()
ind2 = []
for index, row in df_datas.iterrows():
    if "@KLM" in row['text']:
        ind2.append(index)
klm_tweet_men = df_datas.loc[ind2]

In [13]:
klm_tweet_men.head()

,index,created_at,id,id_str,text,display_text_range,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,...,filter_level,lang,timestamp_ms,retweeted_status,extended_tweet,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,delete
54,54,2019-05-22 12:24:26+00:00,1.131174e+18,1.131174e+18,RT @McWhirterAlex: .@KLM will fly Amsterdam-Ba...,NaN,"<a href=""http://twitter.com/download/iphone"" r...",0.0,NaN,NaN,...,low,en,2019-05-22 12:24:26.336,{'created_at': 'Wed May 22 11:49:14 +0000 2019...,NaN,NaN,NaN,NaN,NaN,NaN
55,55,2019-05-22 12:24:31+00:00,1.131174e+18,1.131174e+18,RT @Vinamralongani: @KLM to launch thrice week...,NaN,"<a href=""http://twitter.com/download/iphone"" r...",0.0,NaN,NaN,...,low,en,2019-05-22 12:24:31.580,{'created_at': 'Wed May 22 11:52:42 +0000 2019...,NaN,NaN,NaN,NaN,NaN,NaN
92,92,2019-05-22 12:27:15+00:00,1.131175e+18,1.131175e+18,@wolfmandanny @KLM Thanks Dan,"[19, 29]","<a href=""http://twitter.com/download/iphone"" r...",0.0,1.131164e+18,1.131164e+18,...,low,en,2019-05-22 12:27:15.180,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,173,2019-05-22 12:35:00+00:00,1.131177e+18,1.131177e+18,Waar @KLM de slots vandaan haalt is (nog) niet...,NaN,"<a href=""http://twitter.com/#!/download/ipad"" ...",0.0,NaN,NaN,...,low,nl,2019-05-22 12:35:00.067,NaN,NaN,1.131160e+18,1.131160e+18,{'created_at': 'Wed May 22 11:28:04 +0000 2019...,"{'url': 'https://t.co/GwwWYOYj61', 'expanded':...",NaN
204,204,2019-05-22 12:37:29+00:00,1.131177e+18,1.131177e+18,@KLM Aviation releases about 3% of human carbo...,"[5, 124]","<a href=""https://mobile.twitter.com"" rel=""nofo...",0.0,1.111660e+18,1.111660e+18,...,low,en,2019-05-22 12:37:29.913,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
#Concat 2 tables:
klm_table = pd.concat([klm_tweet, klm_tweet_men])
klm_table.drop_duplicates(subset=['id'], inplace=True)
klm_table.reset_index(inplace=True)

In [62]:
#Add the user_id attribute
klm_user = [klm_table.user.iloc[i].get('id') for i in range(len(klm_table))]
klm_table['user_id'] = pd.Series(klm_user)

In [66]:
#Export the table as CSV file to import to the database
klm_table.to_csv('../database/CSV file/klm_table_diff.csv')

#### 5. Bristish Airways table

In [35]:
#Extract only tweets made by BA (including posting +replying to other):
test = df_datas.copy()

ind = []
for index, row in df_datas.iterrows():
    if row['user']['id_str'] == '18332190':
        ind.append(index)
british_tweet = df_datas.loc[ind]

In [36]:
#Extract only tweets mention BA (@British_Airways etc...):
test2 = df_datas.copy()
ind2 = []
for index, row in df_datas.iterrows():
    if "@British_Airways" in row['text']:
        ind2.append(index)
british_tweet_men = df_datas.loc[ind2]

In [58]:
#Concat 2 tables:
british_table = pd.concat([british_tweet, british_tweet_men])
british_table.drop_duplicates(subset=['id'], inplace=True)
british_table.reset_index(inplace=True)

In [59]:
#Add the user_id attribute
british_user = [british_table.user.iloc[i].get('id') for i in range(len(british_table))]
british_table['user_id'] = pd.Series(british_user)

In [60]:
#Export the table as CSV file to import to the database
british_table.to_csv('../database/CSV file/british_table.csv')

#### 6. Users table 

In [18]:
df_datas.user

0         {'id': 393374091, 'id_str': '393374091', 'name...
1         {'id': 3420691215, 'id_str': '3420691215', 'na...
2         {'id': 394376606, 'id_str': '394376606', 'name...
3         {'id': 36488556, 'id_str': '36488556', 'name':...
4         {'id': 14193348, 'id_str': '14193348', 'name':...
                                ...                        
255012    {'id': 371347123, 'id_str': '371347123', 'name...
255013    {'id': 50015771, 'id_str': '50015771', 'name':...
255014    {'id': 253340062, 'id_str': '253340062', 'name...
255015    {'id': 22536055, 'id_str': '22536055', 'name':...
255016    {'id': 1101932956879335425, 'id_str': '1101932...
Name: user, Length: 255017, dtype: object

In [27]:
#Get a list of dictionary of user information and turn them to dataframe
df_data_user = [df_datas.user.iloc[i] for i in range(len(df_datas))]

KeyboardInterrupt: 

In [32]:
#Turn them to DataFrame and drop the duplicate
user_table = pd.DataFrame(df_data_user).drop_duplicates(subset='id')

In [33]:
user_table.to_csv('../database/CSV file/user_table.csv')